# Import Libraries

In [ ]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import load_model
from sklearn.preprocessing import MinMaxScaler
from datetime import datetime, timedelta

# Download latest stock data

In [ ]:
def download_stock(p, back_days,  Code):

    buffer = 60

    end_date = datetime.now().strftime("%Y-%m-%d")
    start_date = (datetime.now() + timedelta(days = - (p + buffer + back_days))).strftime("%Y-%m-%d")

    data = Download_Stock(Code, start_date, end_date)

    data = data[['Date' , 'Close']]

    return data

# Predict stock price of upcoming X days

In [ ]:
def predict_price(data, model , predicts , p , back_days , scaler):

    full_data = data.tail(p + back_days )
    full_data.reset_index()[['Date', 'Close']]

    full_data['Close_Scaled'] = scaler.fit_transform(full_data[['Close']])


    # Initial Sequence
    Init_df = full_data.iloc[0: p]
    print(f'Start Source From {Init_df.head(1)['Date'].values[0]} to {Init_df.tail(1)['Date'].values[0]}')

    ref_closes = Init_df['Close_Scaled'].tolist()

    pred_prices = []

    for i in range(predicts + back_days):

        ref_price_array = np.array(ref_closes).reshape(-1, 1)
        # scaled_price = scaler.fit_transform(ref_price_array)
        last_sequence = ref_price_array.reshape(1, len(ref_price_array) , 1)
        pred_price = model.predict(last_sequence, verbose=0)[0][0]

        pred_prices.append(pred_price)

        ref_closes.pop(0)

        # Use Actual Price as Next reference, If No, then use the Predicted Price
        try:

            real_price = float(full_data.iloc[i+p]['Close_Scaled'])
            ref_closes.append(real_price)

        except:

            # pred_price = float(scaler.inverse_transform(pred_price.reshape(1,-1))[0][0])

            ref_closes.append(pred_price)




    pred_list = scaler.inverse_transform(np.array(pred_prices).reshape(-1,1))


    return pred_list


# Main

In [ ]:
def main_rnn_predict(Code, num_days , back_days, model, p , scaler):

    data = download_stock(p, back_days , Code)

    pred_list = predict_price(data, model , num_days, p ,back_days , scaler)

    predict_df = Generate_Predict_Date(data, num_days)

    predict_df['Close_Pred'] = np.nan
    predict_df.iloc[-(num_days + back_days):, 2] = pred_list


    title = f"Predict future {num_days} days for Stock {Code} by RNN({p})"

    Plot_Result(predict_df,title,Code)

    return predict_df
